In [2]:
import numpy as np
import bilby
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
from scipy.stats import gaussian_kde
import os

In [3]:
### Units
gcm3_to_dynecm2=8.9875e20 
MeV_to_gcm3 = 1.7827e12 
dynecm2_to_MeVFermi = 1.6022e33
gcm3_to_fm4 = 3.5178e14
sat_dens = 2.8*(np.power(10.0,14.))
c = bilby.core.utils.speed_of_light
c_cgs = c*(1e2)

### Nucleon mass in grams
mass_of_nucleon = 1/(6.02 * 10**23)
### Nucleon mass in MeV/c2
nucleon_mass_in_MeV = 939
### Conversion factors for Mev,fm to CGS units
gcm3_to_dynecm2=8.9875e20 
MeV_to_gcm3 = 1.7827e12 
dynecm2_to_MeVFermi = 1.6022e33
gcm3_to_fm4 = 3.5178e14

fm_in_cgs = 1e-13
rho_nuc_in_cgs = .16 * mass_of_nucleon / (fm_in_cgs)**3

In [4]:
# read h.5 gamma
g_1 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.1_a1._cl.01.h5")
g_3 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.3_a1._cl.01.h5")
g_4 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.4_a1._cl.01.h5")
g_5 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.5_a1._cl.01.h5")
g_7 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._cl.01.h5")
g_8 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.8_a1._cl.01.h5")
g_9 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.9_a1._cl.01.h5")
g_11 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g1.1_a1._cl.01.h5")


In [5]:
# read h.5 alpha
a_001 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a.01_cl.01.h5")
a_005 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a.05_cl.01.h5")
a_01 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a.1_cl.01.h5")
a_03 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a.3_cl.01.h5")
a_05 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a.5_cl.01.h5")
a_1 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._cl.01.h5")
a_5 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a5._cl.01.h5")
a_10 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a10._cl.01.h5")


In [21]:
# read h.5 cl
cl_1 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._cl1..h5")
cl_sr01 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._clsr.1.h5")
cl_01 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._cl.1.h5")
cl_005 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._cl.05.h5")
cl_001 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._cl.01.h5")
cl_0001 = h5py.File("/home/lyla.traylor/agnosticeosmodel/Metamodel+GP/generated_eoss/g.7_a1._cl.001.h5")


In [27]:
cl_01.close()


In [7]:
def get_pe_quantiles(eos_set, eos_data, interp_densities, quantiles, cgs_press = True):
    pdvals = []
    # Obtain quantiles for the prior set of EoS's
    for eos in eos_set:
        eos_densities = eos_data[eos]["baryon_density"]
        eos_pressures = eos_data[eos]["pressurec2"]
        if cgs_press:
            eos_pressures = eos_data[eos]["pressurec2"]*gcm3_to_dynecm2
        pdvals.append(np.interp(interp_densities, eos_densities, eos_pressures))
    pdvals = np.array(pdvals)
    
    # Create object to hold upper and lower bounds 
    pd_sigmas = np.zeros((len(interp_densities),len(quantiles)))
    for i in range(len(interp_densities)):
        pd_sigmas[i]=np.percentile(np.array(pdvals[:,i]),quantiles)
    del pdvals
    return pd_sigmas

In [8]:
def get_mr_quantiles(eos_set, eos_mac_data, interp_masses, quantiles, cutoff = 30., verbose =  False):
    mrvals = []
    num_eos_cut = 0
    for eos in eos_set:
        eos_mass = eos_mac_data[eos]["M"][eos_mac_data[eos]["R"] < cutoff] # cuts to ignore radii values at
        eos_radii = eos_mac_data[eos]["R"][eos_mac_data[eos]["R"] < cutoff] # larger mass from white dwarf branch 
        if len(eos_radii) == 0:
            num_eos_cut += 1
            continue
        ### Try interpolating with scipy function
        massrad_func = interpolate.interp1d(eos_mass, eos_radii, bounds_error = False, fill_value = "NaN")
        interp_rad = massrad_func(interp_masses)
        mrvals.append(interp_rad)
    mrvals = np.array(mrvals)
    
    ## Obtaining quantiles 
    mr_sigmas = np.zeros((len(interp_masses),len(quantiles)))

    ### Expecting NaN's from Eos's that don't reach end mass from interpolation range
    for i in range(len(interp_masses)):
        mr_sigmas[i]=np.nanpercentile(mrvals[:,i],quantiles)

    if verbose:
        print("Number of surviving EoS after radius cuts: ", str(len(eos_set) - num_eos_cut))
    
    del mrvals
    return mr_sigmas

In [9]:
def get_cs_quantiles(eos_set, eos_data, interp_densities, quantiles):
    csvals = []
    # Obtain sound speed quantiles for a set of EoS's
    for eos in eos_set:
        eos_rest_mass = eos_data[eos]["baryon_density"]
        eos_densities = eos_data[eos]["energy_densityc2"]
        eos_pressures = eos_data[eos]["pressurec2"]
        eos_cs = np.gradient(eos_pressures, eos_densities)
        csvals.append(np.interp(interp_densities, eos_rest_mass, eos_cs))
    csvals = np.array(csvals)

    ### Obtaining quantiles 
    cs_sigmas = np.zeros((len(interp_densities),len(quantiles)))
    for i in range(len(interp_densities)):
        cs_sigmas[i] = np.percentile(np.array(csvals[:,i]), quantiles)
    del csvals
    return cs_sigmas


In [10]:

def get_lambda_quantiles(eos_set, eos_mac_data, interp_masses, quantiles, cutoff=30., verbose=False):
    lambda_vals = []
    num_eos_cut = 0

    for eos in eos_set:
        eos_mass = eos_mac_data[eos]["M"][eos_mac_data[eos]["R"] < cutoff]
        eos_lambda = eos_mac_data[eos]["Lambda"][eos_mac_data[eos]["R"] < cutoff]
        if len(eos_lambda) == 0:
            num_eos_cut += 1
            continue

        ### Try interpolating with scipy function
        mass_lambda_func = interpolate.interp1d(eos_mass, eos_lambda, bounds_error=False, fill_value= "NaN")
        interp_lambda = mass_lambda_func(interp_masses)
        lambda_vals.append(interp_lambda)

    lambda_vals = np.array(lambda_vals)
    
    ## Obtaining quantiles
    lambda_sigmas = np.zeros((len(interp_masses), len(quantiles)))
    for i in range(len(interp_masses)):
        lambda_sigmas[i] = np.nanpercentile(lambda_vals[:, i], quantiles)

    if verbose:
        print("Number of surviving EoS after radius cuts: ", str(len(eos_set) - num_eos_cut))

    del lambda_vals
    return lambda_sigmas


In [11]:

def get_max_mass_quantiles(eos_set, eos_mac_data, quantiles, cutoff = 3.0):
    max_masses = []
    num_eos_cut = 0

    for eos in eos_set:
        eos_mass = eos_mac_data[eos]["M"]
        if len(eos_mass) == 0:
            continue
        max_mass = np.nanmax(eos_mass)
        max_masses.append(max_mass)

    max_masses = np.array(max_masses)
    mass_quantiles = np.nanpercentile(max_masses, quantiles)

    return mass_quantiles

In [17]:
plot_densities = np.geomspace(np.power(10.0,13.0), 10.*rho_nuc_in_cgs, 5000)  #interpolatin range
plot_masses = np.linspace(0.5, 3.5, 5000)
percentiles = [50,95] # list of values

In [18]:
p_sigmas = get_pe_quantiles(eos_to_be_used, micro_data, plot_densities, percentiles)
int_func = interplate.interpld(plot_densities, p_sigmas[:,1])
int_funct(2*rho_nuc_in_cgs)

NameError: name 'eos_to_be_used' is not defined

In [19]:
micro_g7 = {eos_num : np.array(g_7['eos'][eos_id]) for eos_num, eos_id in enumerate(g_7['eos'])}
macro_g7 = {eos_num : np.array(g_7['ns'][eos_id]) for eos_num, eos_id in enumerate(g_7['ns'])}

eos_to_be_used = np.arange(len(g_7["eos"]))

plot_masses = np.linspace(0.5, 3.5, 5000)
plot_densities = np.geomspace(np.power(10.0,13.0), 10.*rho_nuc_in_cgs, 5000)  #interpolatin range

percentiles = [16,84] # list of values

In [20]:
p_sigmas_g7 = get_pe_quantiles(eos_to_be_used, micro_g7, plot_densities, percentiles)
mr_sigmas_g7 = get_mr_quantiles(eos_to_be_used, macro_g7, plot_masses, percentiles, cutoff = 30., verbose =  False)
cs_sigmas_g7 = get_cs_quantiles(eos_to_be_used, micro_g7, plot_densities, [50,90])

In [22]:
int_func_cs = int_func = interpolate.interp1d(plot_densities, cs_sigmas_g7[:,0])
int_func(6*rho_nuc_in_cgs)

array(0.42492387)

In [60]:
int_func = interpolate.interp1d(plot_densities, p_sigmas_g7[:,0])
int_func(2*rho_nuc_in_cgs)

array(8.98722712e+33)

In [28]:
int_func(6*rho_nuc_in_cgs)

array(4.83666258e+35)

In [62]:
int_func_m(1.5)

array(13.06619702)

In [12]:
def diagnose_eos_set(eos_set, percentile = [5, 95], verbose = True):
    eos_to_be_used = np.arange(len(eos_set["eos"]))
    eos_to_be_used_ns = np.arange(len(eos_set["ns"]))
    percentiles_median = [50,95];
    percentiles_quartile = percentile;

    # interpolate over
    plot_densities = np.geomspace(np.power(10.0,13.0), 10.*rho_nuc_in_cgs, 5000)
    plot_masses = np.linspace(0.5, 3.5, 5000)

    # correct enumerations
    micro = {eos_num : np.array(eos_set['eos'][eos_id]) for eos_num, eos_id in enumerate(eos_set['eos'])}
    macro = {eos_num : np.array(eos_set['ns'][eos_id]) for eos_num, eos_id in enumerate(eos_set['ns'])}

    # pe to get mean
    p_sigmas_median = get_pe_quantiles(eos_to_be_used, micro, plot_densities, percentiles_median)
    int_func_median = interpolate.interp1d(plot_densities, p_sigmas_median[:,0])
    p_at2 = int_func_median(2*rho_nuc_in_cgs) # val at 2
    p_at6 = int_func_median(6*rho_nuc_in_cgs) # val at 6

    # pe to get quartile
    p_sigmas_qt = get_pe_quantiles(eos_to_be_used, micro, plot_densities, percentiles_quartile)
    int_func_lower = interpolate.interp1d(plot_densities, p_sigmas_qt[:,0])
    int_func_upper = interpolate.interp1d(plot_densities, p_sigmas_qt[:,1])
    p_lower2 = p_at2 - int_func_lower(2*rho_nuc_in_cgs) 
    p_upper2 = int_func_upper(2*rho_nuc_in_cgs) - p_at2
    p_lower6 = p_at6 - int_func_lower(6*rho_nuc_in_cgs)
    p_upper6 = int_func_upper(6*rho_nuc_in_cgs) - p_at6

    # mr to get mean
    mr_sigmas_median = get_mr_quantiles(eos_to_be_used_ns, macro, plot_masses, percentiles_median, cutoff = 30., verbose =  False)
    int_func_m_median = interpolate.interp1d(plot_masses, mr_sigmas_median[:,0])
    median_radi = int_func_m_median(1.4)
    median_radi2 = int_func_m_median(2)

    # mr to get qt
    mr_sigmas_qt = get_mr_quantiles(eos_to_be_used_ns, macro, plot_masses, percentiles_quartile, cutoff = 30., verbose =  False)
    int_func_m_lower = interpolate.interp1d(plot_masses, mr_sigmas_qt[:,0])
    int_func_m_upper = interpolate.interp1d(plot_masses, mr_sigmas_qt[:,1])
    lower_radi = median_radi - int_func_m_lower(1.4)
    upper_radi = int_func_m_upper(1.4) - median_radi
    lower_radi2 = median_radi2 - int_func_m_lower(2)
    upper_radi2 = int_func_m_upper(2) - median_radi2

    # se to get mean
    cs_sigmas_median = get_cs_quantiles(eos_to_be_used, micro, plot_densities, percentiles_median)
    int_func_cs_median = interpolate.interp1d(plot_densities, cs_sigmas_median[:,0])
    cs_at2 = int_func_cs_median(2*rho_nuc_in_cgs) # val at 2
    cs_at6 = int_func_cs_median(6*rho_nuc_in_cgs) # val at 6

    # pe to get quartile
    cs_sigmas_qt = get_cs_quantiles(eos_to_be_used, micro, plot_densities, percentiles_quartile)
    int_func_lower_cs = interpolate.interp1d(plot_densities, cs_sigmas_qt[:,0])
    int_func_upper_cs = interpolate.interp1d(plot_densities, cs_sigmas_qt[:,1])
    cs_lower2 = cs_at2 - int_func_lower_cs(2*rho_nuc_in_cgs) 
    cs_upper2 = int_func_upper_cs(2*rho_nuc_in_cgs) - cs_at2
    cs_lower6 = cs_at6 - int_func_lower_cs(6*rho_nuc_in_cgs)
    cs_upper6 = int_func_upper_cs(6*rho_nuc_in_cgs) - cs_at6

    # l to get mean
    l_sigmas_median = get_lambda_quantiles(eos_to_be_used_ns, macro, plot_masses, percentiles_median, cutoff = 30., verbose =  False)
    int_func_l_median = interpolate.interp1d(plot_masses, l_sigmas_median[:,0])
    median_lam = int_func_l_median(1.4)

    # l to get qt
    l_sigmas_qt = get_lambda_quantiles(eos_to_be_used_ns, macro, plot_masses, percentiles_quartile, cutoff = 30., verbose =  False)
    int_func_l_lower = interpolate.interp1d(plot_masses, l_sigmas_qt[:,0])
    int_func_l_upper = interpolate.interp1d(plot_masses, l_sigmas_qt[:,1])
    lower_lam = median_lam - int_func_l_lower(1.4)
    upper_lam = int_func_l_upper(1.4) - median_lam

    # (median) max mass
    med_max_mass = get_max_mass_quantiles(eos_to_be_used_ns, macro, [50], cutoff = 30.)
    # radius at that mass
    median_radi_max = int_func_m_median(med_max_mass[0])
    lower_radi_max = median_radi_max - int_func_m_lower(med_max_mass[0])
    upper_radi_max = int_func_m_upper(med_max_mass[0]) - median_radi_max

    
    if verbose:
        print(f"For the set {eos_set}: \n" 
                f"Median pressure at 2x nuclear saturation density- {p_at2} \n"
                f"Lower \aand upper bound for quartile pressure at 2x nuclear saturation density- {p_lower2}, {p_upper2} \n"
                f"Median pressure at 6x nuclear saturation density- {p_at6} \n"
                f"Lower \aand upper bound for quartile pressure at 6x nuclear saturation density- {p_lower6}, {p_upper6} \n"
                f"Median radius at 1.4 solar masses {median_radi} \n"
                f"Lower \aand upper bound for quartile radius at 1.4 solar masses- {lower_radi}, {upper_radi} \n"
                f"Median radius at 2 solar masses {median_radi2} \n"
                f"Lower \aand upper bound for quartile radius at 2 solar masses- {lower_radi2}, {upper_radi2} \n"
                f"Median max mass {med_max_mass[0]} \n"
                f"Median radius at median max mass {median_radi_max} \n"
                f"Lower \aand upper bound for quartile radius at median max mass- {lower_radi_max}, {upper_radi_max} \n"
                f"Median sound speed at 2x nuclear saturation density- {cs_at2} \n"
                f"Lower \aand upper bound for quartile sound speed at 2x nuclear saturation density- {cs_lower2}, {cs_upper2} \n"
                f"Median sound speed at 6x nuclear saturation density- {cs_at6} \n"
                f"Lower \aand upper bound for quartile sound speed at 6x nuclear saturation density- {cs_lower6}, {cs_upper6} \n"
                f"Median lambda at 1.4 solar masses {median_lam} \n"
                f"Lower \aand upper bound for quartile lambda at 1.4 solar masses- {lower_lam}, {upper_lam}")
    return {"pressure_2sat": p_at2, "p_lower_2sat": p_lower2, "p_upper_2sat": p_upper2,
            "pressure_6sat": p_at6, "p_lower_6sat": p_lower6, "p_upper_6sat": p_upper6,
            "radius_1.4":  median_radi, "radius_lower_1.4":lower_radi, "radius_upper_1.4": upper_radi,
            "radius_2":  median_radi2, "radius_lower_2":lower_radi2, "radius_upper_2": upper_radi2,
            "median_max_mass": med_max_mass[0],
            "radius_max":  median_radi_max, "radius_lower_max":lower_radi_max, "radius_upper_max": upper_radi_max,
            "cs_2sat": cs_at2, "cs_lower_2sat": cs_lower2, "cs_upper_2sat": cs_upper2,
            "cs_6sat": cs_at6, "cs_lower_6sat": cs_lower6, "cs_upper_6sat": cs_upper6,
            "lambda":  median_lam, "lambda_lower":lower_lam, "lambda_upper": upper_lam}


In [72]:
#def control_compare(eos_set):
    #[A, B, C, D, E, F, G, H ,I] = get_numbers(g_7)
    #[a, b, c, d, e, f, g, h ,i] = get_numbers(eos_set)
    #return [A-a, B-b, C-c, D-d, E-e, F-f, G-g, H-h ,I-i]
    
    

In [13]:
diagnose_eos_set(g_7, verbose = True)

For the set <HDF5 file "g.7_a1._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.1657425287981534e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.65400195286234e+34, 1.547892560504343e+35 
Median pressure at 6x nuclear saturation density- 4.836662576167329e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 4.649542032617266e+35, 1.9807267806850263e+36 
Median radius at 1.4 solar masses 12.94570439031916 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.94525781247831, 1.44556764891297 
Median radius at 2 solar masses 13.79773989129582 
Lower and upper bound for quartile radius at 2 solar masses- 3.2560524210952018, 1.4724380608383658 
Median max mass 2.0398771477880606 
Median radius at median max mass 13.858821837453076 
Lower and upper bound for quartile radius at median max mass- 3.1380450659318164, 1.4629695185527591 
Median sound speed at 2x nuclear saturation density

{'pressure_2sat': array(3.16574253e+34),
 'p_lower_2sat': 2.65400195286234e+34,
 'p_upper_2sat': 1.547892560504343e+35,
 'pressure_6sat': array(4.83666258e+35),
 'p_lower_6sat': 4.649542032617266e+35,
 'p_upper_6sat': 1.9807267806850263e+36,
 'radius_1.4': array(12.94570439),
 'radius_lower_1.4': 3.94525781247831,
 'radius_upper_1.4': 1.44556764891297,
 'radius_2': array(13.79773989),
 'radius_lower_2': 3.2560524210952018,
 'radius_upper_2': 1.4724380608383658,
 'median_max_mass': 2.0398771477880606,
 'radius_max': array(13.85882184),
 'radius_lower_max': 3.1380450659318164,
 'radius_upper_max': 1.4629695185527591,
 'cs_2sat': array(0.14537894),
 'cs_lower_2sat': 0.1431299131454666,
 'cs_upper_2sat': 0.7814083051822388,
 'cs_6sat': array(0.42492387),
 'cs_lower_6sat': 0.41991829230465993,
 'cs_upper_6sat': 0.5663360319973965,
 'lambda': array(667.82494792),
 'lambda_lower': 621.5706902823191,
 'lambda_upper': 715.5602486998844}

In [14]:
diagnose_eos_set(g_1)

/opt/conda/igwn-py310-20250513/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


For the set <HDF5 file "g.1_a1._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.0000336585392815e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.0680191776037046e+34, 2.6559610134997453e+34 
Median pressure at 6x nuclear saturation density- 3.963270123031499e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 3.0226575970039226e+35, 4.420638666771308e+35 
Median radius at 1.4 solar masses 12.169954462127778 
Lower and upper bound for quartile radius at 1.4 solar masses- 2.405392975328736, 1.3561623781104615 
Median radius at 2 solar masses 12.580550444839666 
Lower and upper bound for quartile radius at 2 solar masses- 1.4112434771557663, 1.2753885347671492 
Median max mass 1.7024513160968366 
Median radius at median max mass 12.254791097484325 
Lower and upper bound for quartile radius at median max mass- 1.9212890573322028, 1.4194241831514631 
Median sound speed at 2x nuclear saturation

{'pressure_2sat': array(3.00003366e+34),
 'p_lower_2sat': 2.0680191776037046e+34,
 'p_upper_2sat': 2.6559610134997453e+34,
 'pressure_6sat': array(3.96327012e+35),
 'p_lower_6sat': 3.0226575970039226e+35,
 'p_upper_6sat': 4.420638666771308e+35,
 'radius_1.4': array(12.16995446),
 'radius_lower_1.4': 2.405392975328736,
 'radius_upper_1.4': 1.3561623781104615,
 'radius_2': array(12.58055044),
 'radius_lower_2': 1.4112434771557663,
 'radius_upper_2': 1.2753885347671492,
 'median_max_mass': 1.7024513160968366,
 'radius_max': array(12.2547911),
 'radius_lower_max': 1.9212890573322028,
 'radius_upper_max': 1.4194241831514631,
 'cs_2sat': array(0.15130099),
 'cs_lower_2sat': 0.1106623938344585,
 'cs_upper_2sat': 0.15482588566283076,
 'cs_6sat': array(0.41698792),
 'cs_lower_6sat': 0.3103455906180441,
 'cs_upper_6sat': 0.2714192640089588,
 'lambda': array(380.93376102),
 'lambda_lower': 311.1480734469204,
 'lambda_upper': 456.43093797400235}

In [15]:
diagnose_eos_set(g_3)

For the set <HDF5 file "g.3_a1._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 2.9531121011943864e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.1736316249414507e+34, 6.207571178512777e+34 
Median pressure at 6x nuclear saturation density- 4.177512944356824e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 3.5429608551723595e+35, 1.0125716254752399e+36 
Median radius at 1.4 solar masses 12.421205887979575 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.109779204846289, 1.5314826460689197 
Median radius at 2 solar masses 13.137334780208555 
Lower and upper bound for quartile radius at 2 solar masses- 2.3422015023232134, 1.4914671932476047 
Median max mass 1.7909000401492774 
Median radius at median max mass 12.833339107606276 
Lower and upper bound for quartile radius at median max mass- 2.693188730559724, 1.5571441237037877 
Median sound speed at 2x nuclear saturation 

{'pressure_2sat': array(2.9531121e+34),
 'p_lower_2sat': 2.1736316249414507e+34,
 'p_upper_2sat': 6.207571178512777e+34,
 'pressure_6sat': array(4.17751294e+35),
 'p_lower_6sat': 3.5429608551723595e+35,
 'p_upper_6sat': 1.0125716254752399e+36,
 'radius_1.4': array(12.42120589),
 'radius_lower_1.4': 3.109779204846289,
 'radius_upper_1.4': 1.5314826460689197,
 'radius_2': array(13.13733478),
 'radius_lower_2': 2.3422015023232134,
 'radius_upper_2': 1.4914671932476047,
 'median_max_mass': 1.7909000401492774,
 'radius_max': array(12.83333911),
 'radius_lower_max': 2.693188730559724,
 'radius_upper_max': 1.5571441237037877,
 'cs_2sat': array(0.14745221),
 'cs_lower_2sat': 0.12675526936405238,
 'cs_upper_2sat': 0.40272634184455375,
 'cs_6sat': array(0.43529966),
 'cs_lower_6sat': 0.37226068711231525,
 'cs_upper_6sat': 0.4489330771166565,
 'lambda': array(468.27933463),
 'lambda_lower': 416.71548343535767,
 'lambda_upper': 615.1497416806068}

In [16]:
diagnose_eos_set(g_5)

For the set <HDF5 file "g.5_a1._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.0384312409543436e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.42722956912598e+34, 1.1229148635062062e+35 
Median pressure at 6x nuclear saturation density- 4.49319613905129e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 4.164825967005292e+35, 1.57716486509034e+36 
Median radius at 1.4 solar masses 12.748244105712981 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.6476423288691233, 1.4722932646609195 
Median radius at 2 solar masses 13.532071510238548 
Lower and upper bound for quartile radius at 2 solar masses- 2.862975759873928, 1.5010456828464918 
Median max mass 1.9119962948889166 
Median radius at median max mass 13.401644829814579 
Lower and upper bound for quartile radius at median max mass- 3.0198126441251674, 1.51892842737427 
Median sound speed at 2x nuclear saturation densit

{'pressure_2sat': array(3.03843124e+34),
 'p_lower_2sat': 2.42722956912598e+34,
 'p_upper_2sat': 1.1229148635062062e+35,
 'pressure_6sat': array(4.49319614e+35),
 'p_lower_6sat': 4.164825967005292e+35,
 'p_upper_6sat': 1.57716486509034e+36,
 'radius_1.4': array(12.74824411),
 'radius_lower_1.4': 3.6476423288691233,
 'radius_upper_1.4': 1.4722932646609195,
 'radius_2': array(13.53207151),
 'radius_lower_2': 2.862975759873928,
 'radius_upper_2': 1.5010456828464918,
 'median_max_mass': 1.9119962948889166,
 'radius_max': array(13.40164483),
 'radius_lower_max': 3.0198126441251674,
 'radius_upper_max': 1.51892842737427,
 'cs_2sat': array(0.14354841),
 'cs_lower_2sat': 0.13630301306602027,
 'cs_upper_2sat': 0.6548452797037383,
 'cs_6sat': array(0.43192441),
 'cs_lower_6sat': 0.4136231305252183,
 'cs_upper_6sat': 0.535493540031738,
 'lambda': array(587.5897097),
 'lambda_lower': 540.2137630945288,
 'lambda_upper': 674.3857565711335}

In [17]:
diagnose_eos_set(g_9)

For the set <HDF5 file "g.9_a1._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.4867599418522813e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 3.0366450130528173e+34, 1.8739166007091006e+35 
Median pressure at 6x nuclear saturation density- 5.446830855187488e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 5.320744357736831e+35, 2.2642898562377702e+36 
Median radius at 1.4 solar masses 13.158107727974564 
Lower and upper bound for quartile radius at 1.4 solar masses- 4.292714390271788, 1.3489402383893818 
Median radius at 2 solar masses 14.04643629124429 
Lower and upper bound for quartile radius at 2 solar masses- 3.5123288605468517, 1.3812976560019816 
Median max mass 2.1652209216060347 
Median radius at median max mass 14.299404176837982 
Lower and upper bound for quartile radius at median max mass- 3.181459620198238, 1.355077437514522 
Median sound speed at 2x nuclear saturation de

{'pressure_2sat': array(3.48675994e+34),
 'p_lower_2sat': 3.0366450130528173e+34,
 'p_upper_2sat': 1.8739166007091006e+35,
 'pressure_6sat': array(5.44683086e+35),
 'p_lower_6sat': 5.320744357736831e+35,
 'p_upper_6sat': 2.2642898562377702e+36,
 'radius_1.4': array(13.15810773),
 'radius_lower_1.4': 4.292714390271788,
 'radius_upper_1.4': 1.3489402383893818,
 'radius_2': array(14.04643629),
 'radius_lower_2': 3.5123288605468517,
 'radius_upper_2': 1.3812976560019816,
 'median_max_mass': 2.1652209216060347,
 'radius_max': array(14.29940418),
 'radius_lower_max': 3.181459620198238,
 'radius_upper_max': 1.355077437514522,
 'cs_2sat': array(0.14492702),
 'cs_lower_2sat': 0.14420348442267542,
 'cs_upper_2sat': 0.8329679501855408,
 'cs_6sat': array(0.44405188),
 'cs_lower_6sat': 0.4426970738306495,
 'cs_upper_6sat': 0.5537426207742591,
 'lambda': array(756.50568656),
 'lambda_lower': 712.0917308679598,
 'lambda_upper': 718.7156435532614}

In [18]:
diagnose_eos_set(a_10)

For the set <HDF5 file "g.7_a10._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.0960007966907106e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.620156195492989e+34, 1.7048928398415794e+35 
Median pressure at 6x nuclear saturation density- 5.063551935790229e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 4.883454155130763e+35, 2.0504745032950576e+36 
Median radius at 1.4 solar masses 12.926530762616506 
Lower and upper bound for quartile radius at 1.4 solar masses- 4.02691750826029, 1.4789052499815831 
Median radius at 2 solar masses 13.826043739109824 
Lower and upper bound for quartile radius at 2 solar masses- 3.3308970616391225, 1.4669230444832753 
Median max mass 2.085840917811785 
Median radius at median max mass 13.960659523366353 
Lower and upper bound for quartile radius at median max mass- 3.2163239665645023, 1.4472816321595374 
Median sound speed at 2x nuclear saturation d

{'pressure_2sat': array(3.0960008e+34),
 'p_lower_2sat': 2.620156195492989e+34,
 'p_upper_2sat': 1.7048928398415794e+35,
 'pressure_6sat': array(5.06355194e+35),
 'p_lower_6sat': 4.883454155130763e+35,
 'p_upper_6sat': 2.0504745032950576e+36,
 'radius_1.4': array(12.92653076),
 'radius_lower_1.4': 4.02691750826029,
 'radius_upper_1.4': 1.4789052499815831,
 'radius_2': array(13.82604374),
 'radius_lower_2': 3.3308970616391225,
 'radius_upper_2': 1.4669230444832753,
 'median_max_mass': 2.085840917811785,
 'radius_max': array(13.96065952),
 'radius_lower_max': 3.2163239665645023,
 'radius_upper_max': 1.4472816321595374,
 'cs_2sat': array(0.1406141),
 'cs_lower_2sat': 0.13922278453080536,
 'cs_upper_2sat': 0.813244892583277,
 'cs_6sat': array(0.42974156),
 'cs_lower_6sat': 0.4256068698205425,
 'cs_upper_6sat': 0.5625737958762692,
 'lambda': array(666.76588577),
 'lambda_lower': 623.146110445163,
 'lambda_upper': 733.730710154697}

In [19]:
diagnose_eos_set(a_5)

For the set <HDF5 file "g.7_a5._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.226578953281065e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.7426772667809676e+34, 1.6529398419682158e+35 
Median pressure at 6x nuclear saturation density- 5.221207812291147e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 5.027729720020024e+35, 2.0137226655949133e+36 
Median radius at 1.4 solar masses 12.929060334647412 
Lower and upper bound for quartile radius at 1.4 solar masses- 4.026134323774597, 1.4783729680791087 
Median radius at 2 solar masses 13.806084630104136 
Lower and upper bound for quartile radius at 2 solar masses- 3.302432888184047, 1.4835930974042935 
Median max mass 2.082989666527058 
Median radius at median max mass 13.948142358138695 
Lower and upper bound for quartile radius at median max mass- 3.1454528326701485, 1.4578547372239683 
Median sound speed at 2x nuclear saturation de

{'pressure_2sat': array(3.22657895e+34),
 'p_lower_2sat': 2.7426772667809676e+34,
 'p_upper_2sat': 1.6529398419682158e+35,
 'pressure_6sat': array(5.22120781e+35),
 'p_lower_6sat': 5.027729720020024e+35,
 'p_upper_6sat': 2.0137226655949133e+36,
 'radius_1.4': array(12.92906033),
 'radius_lower_1.4': 4.026134323774597,
 'radius_upper_1.4': 1.4783729680791087,
 'radius_2': array(13.80608463),
 'radius_lower_2': 3.302432888184047,
 'radius_upper_2': 1.4835930974042935,
 'median_max_mass': 2.082989666527058,
 'radius_max': array(13.94814236),
 'radius_lower_max': 3.1454528326701485,
 'radius_upper_max': 1.4578547372239683,
 'cs_2sat': array(0.1475287),
 'cs_lower_2sat': 0.1460374712320692,
 'cs_upper_2sat': 0.8038214281414275,
 'cs_6sat': array(0.44044615),
 'cs_lower_6sat': 0.43595979208621427,
 'cs_upper_6sat': 0.5523963360853088,
 'lambda': array(668.87387184),
 'lambda_lower': 625.4515721200063,
 'lambda_upper': 726.0720186577446}

In [24]:
diagnose_eos_set(a_1)

For the set <HDF5 file "g.7_a1._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.1657425287981534e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.65400195286234e+34, 1.547892560504343e+35 
Median pressure at 6x nuclear saturation density- 4.836662576167329e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 4.649542032617266e+35, 1.9807267806850263e+36 
Median radius at 1.4 solar masses 12.94570439031916 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.94525781247831, 1.44556764891297 
Median radius at 2 solar masses 13.79773989129582 
Lower and upper bound for quartile radius at 2 solar masses- 3.2560524210952018, 1.4724380608383658 
Median max mass 2.0398771477880606 
Median radius at median max mass 13.858821837453076 
Lower and upper bound for quartile radius at median max mass- 3.1380450659318164, 1.4629695185527591 
Median sound speed at 2x nuclear saturation density

{'pressure_2sat': array(3.16574253e+34),
 'p_lower_2sat': 2.65400195286234e+34,
 'p_upper_2sat': 1.547892560504343e+35,
 'pressure_6sat': array(4.83666258e+35),
 'p_lower_6sat': 4.649542032617266e+35,
 'p_upper_6sat': 1.9807267806850263e+36,
 'radius_1.4': array(12.94570439),
 'radius_lower_1.4': 3.94525781247831,
 'radius_upper_1.4': 1.44556764891297,
 'radius_2': array(13.79773989),
 'radius_lower_2': 3.2560524210952018,
 'radius_upper_2': 1.4724380608383658,
 'median_max_mass': 2.0398771477880606,
 'radius_max': array(13.85882184),
 'radius_lower_max': 3.1380450659318164,
 'radius_upper_max': 1.4629695185527591,
 'cs_2sat': array(0.14537894),
 'cs_lower_2sat': 0.1431299131454666,
 'cs_upper_2sat': 0.7814083051822388,
 'cs_6sat': array(0.42492387),
 'cs_lower_6sat': 0.41991829230465993,
 'cs_upper_6sat': 0.5663360319973965,
 'lambda': array(667.82494792),
 'lambda_lower': 621.5706902823191,
 'lambda_upper': 715.5602486998844}

In [25]:
diagnose_eos_set(a_05)

For the set <HDF5 file "g.7_a.5_cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.2582347162508434e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.7219330311235776e+34, 1.4199025286657067e+35 
Median pressure at 6x nuclear saturation density- 4.8549318062202254e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 4.6338933720821266e+35, 1.8744827344243522e+36 
Median radius at 1.4 solar masses 12.928090391055976 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.920160737019021, 1.4400077085294978 
Median radius at 2 solar masses 13.760263352046106 
Lower and upper bound for quartile radius at 2 solar masses- 3.1576020249961196, 1.4735521254087924 
Median max mass 2.0187142920526053 
Median radius at median max mass 13.793241517060597 
Lower and upper bound for quartile radius at median max mass- 3.1357603767557585, 1.4660360429144408 
Median sound speed at 2x nuclear saturati

{'pressure_2sat': array(3.25823472e+34),
 'p_lower_2sat': 2.7219330311235776e+34,
 'p_upper_2sat': 1.4199025286657067e+35,
 'pressure_6sat': array(4.85493181e+35),
 'p_lower_6sat': 4.6338933720821266e+35,
 'p_upper_6sat': 1.8744827344243522e+36,
 'radius_1.4': array(12.92809039),
 'radius_lower_1.4': 3.920160737019021,
 'radius_upper_1.4': 1.4400077085294978,
 'radius_2': array(13.76026335),
 'radius_lower_2': 3.1576020249961196,
 'radius_upper_2': 1.4735521254087924,
 'median_max_mass': 2.0187142920526053,
 'radius_max': array(13.79324152),
 'radius_lower_max': 3.1357603767557585,
 'radius_upper_max': 1.4660360429144408,
 'cs_2sat': array(0.14841961),
 'cs_lower_2sat': 0.14513176653260382,
 'cs_upper_2sat': 0.7492907159306343,
 'cs_6sat': array(0.44058687),
 'cs_lower_6sat': 0.43378058671662567,
 'cs_upper_6sat': 0.5472639674614104,
 'lambda': array(654.97232144),
 'lambda_lower': 609.4876253671998,
 'lambda_upper': 713.9700300193193}

In [14]:
diagnose_eos_set(a_01)

For the set <HDF5 file "g.7_a.1_cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.1304057286275134e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.4752804540477423e+34, 9.676098485606453e+34 
Median pressure at 6x nuclear saturation density- 4.432417108956113e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 4.0373145331284744e+35, 1.4029478645930917e+36 
Median radius at 1.4 solar masses 12.753323260953007 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.4825310024046523, 1.5001406091843599 
Median radius at 2 solar masses 13.52296608831713 
Lower and upper bound for quartile radius at 2 solar masses- 2.701722949366955, 1.53451389216867 
Median max mass 1.9117522913512597 
Median radius at median max mass 13.389031741000212 
Lower and upper bound for quartile radius at median max mass- 2.8642445854972305, 1.5558894445357954 
Median sound speed at 2x nuclear saturation de

{'pressure_2sat': array(3.13040573e+34),
 'p_lower_2sat': 2.4752804540477423e+34,
 'p_upper_2sat': 9.676098485606453e+34,
 'pressure_6sat': array(4.43241711e+35),
 'p_lower_6sat': 4.0373145331284744e+35,
 'p_upper_6sat': 1.4029478645930917e+36,
 'radius_1.4': array(12.75332326),
 'radius_lower_1.4': 3.4825310024046523,
 'radius_upper_1.4': 1.5001406091843599,
 'radius_2': array(13.52296609),
 'radius_lower_2': 2.701722949366955,
 'radius_upper_2': 1.53451389216867,
 'median_max_mass': 1.9117522913512597,
 'radius_max': array(13.38903174),
 'radius_lower_max': 2.8642445854972305,
 'radius_upper_max': 1.5558894445357954,
 'cs_2sat': array(0.14506842),
 'cs_lower_2sat': 0.13576559039882508,
 'cs_upper_2sat': 0.5795280354752734,
 'cs_6sat': array(0.43120341),
 'cs_lower_6sat': 0.4063433950950421,
 'cs_upper_6sat': 0.5198343508448133,
 'lambda': array(583.10312592),
 'lambda_lower': 530.4312227882277,
 'lambda_upper': 709.7408700124776}

In [28]:
diagnose_eos_set(a_001)

For the set <HDF5 file "g.7_a.01_cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.026519232337924e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.150659013731641e+34, 4.170021428723583e+34 
Median pressure at 6x nuclear saturation density- 4.19903670947722e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 3.2052788434517338e+35, 6.380674339813248e+35 
Median radius at 1.4 solar masses 12.300382778997239 
Lower and upper bound for quartile radius at 1.4 solar masses- 2.724651787599907, 1.5472462364281494 
Median radius at 2 solar masses 12.931605738326436 
Lower and upper bound for quartile radius at 2 solar masses- 1.8255888310109523, 1.510361463451579 
Median max mass 1.7571677552422185 
Median radius at median max mass 12.555864134745454 
Lower and upper bound for quartile radius at median max mass- 2.226383231807624, 1.6221904957349587 
Median sound speed at 2x nuclear saturation dens

{'pressure_2sat': array(3.02651923e+34),
 'p_lower_2sat': 2.150659013731641e+34,
 'p_upper_2sat': 4.170021428723583e+34,
 'pressure_6sat': array(4.19903671e+35),
 'p_lower_6sat': 3.2052788434517338e+35,
 'p_upper_6sat': 6.380674339813248e+35,
 'radius_1.4': array(12.30038278),
 'radius_lower_1.4': 2.724651787599907,
 'radius_upper_1.4': 1.5472462364281494,
 'radius_2': array(12.93160574),
 'radius_lower_2': 1.8255888310109523,
 'radius_upper_2': 1.510361463451579,
 'median_max_mass': 1.7571677552422185,
 'radius_max': array(12.55586413),
 'radius_lower_max': 2.226383231807624,
 'radius_upper_max': 1.6221904957349587,
 'cs_2sat': array(0.14941162),
 'cs_lower_2sat': 0.11695253007037879,
 'cs_upper_2sat': 0.2575806418299111,
 'cs_6sat': array(0.44204618),
 'cs_lower_6sat': 0.325690880692733,
 'cs_upper_6sat': 0.3446443983847603,
 'lambda': array(422.48981938),
 'lambda_lower': 361.45018495652687,
 'lambda_upper': 588.5567092449974}

In [30]:
diagnose_eos_set(a_005)

For the set <HDF5 file "g.7_a.05_cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.0955196624787886e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.372100351801327e+34, 7.861937740247899e+34 
Median pressure at 6x nuclear saturation density- 4.308588090423983e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 3.744198147067326e+35, 1.1481733522515826e+36 
Median radius at 1.4 solar masses 12.639987751063483 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.274456655024986, 1.5297664380822162 
Median radius at 2 solar masses 13.37202788582408 
Lower and upper bound for quartile radius at 2 solar masses- 2.461097119143094, 1.5644216843017364 
Median max mass 1.8274638197618 
Median radius at median max mass 13.119492749493144 
Lower and upper bound for quartile radius at median max mass- 2.743111940756039, 1.6051247648131248 
Median sound speed at 2x nuclear saturation densit

{'pressure_2sat': array(3.09551966e+34),
 'p_lower_2sat': 2.372100351801327e+34,
 'p_upper_2sat': 7.861937740247899e+34,
 'pressure_6sat': array(4.30858809e+35),
 'p_lower_6sat': 3.744198147067326e+35,
 'p_upper_6sat': 1.1481733522515826e+36,
 'radius_1.4': array(12.63998775),
 'radius_lower_1.4': 3.274456655024986,
 'radius_upper_1.4': 1.5297664380822162,
 'radius_2': array(13.37202789),
 'radius_lower_2': 2.461097119143094,
 'radius_upper_2': 1.5644216843017364,
 'median_max_mass': 1.8274638197618,
 'radius_max': array(13.11949275),
 'radius_lower_max': 2.743111940756039,
 'radius_upper_max': 1.6051247648131248,
 'cs_2sat': array(0.14640174),
 'cs_lower_2sat': 0.13072760770271874,
 'cs_upper_2sat': 0.4771317624662044,
 'cs_6sat': array(0.43278379),
 'cs_lower_6sat': 0.38650352364873997,
 'cs_upper_6sat': 0.48332364221745955,
 'lambda': array(535.21502956),
 'lambda_lower': 480.642995676945,
 'lambda_upper': 692.6369578251944}

In [22]:
diagnose_eos_set(cl_0001) #cl_0001 , cl_001, cl_005, cl_01, cl_sr01, cl_1

KeyError: "Unable to synchronously open object (object 'ns' doesn't exist)"

In [27]:
diagnose_eos_set(cl_001)

For the set <HDF5 file "g.7_a1._cl.01.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.1657425287981534e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.65400195286234e+34, 1.547892560504343e+35 
Median pressure at 6x nuclear saturation density- 4.836662576167329e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 4.649542032617266e+35, 1.9807267806850263e+36 
Median radius at 1.4 solar masses 12.94570439031916 
Lower and upper bound for quartile radius at 1.4 solar masses- 3.94525781247831, 1.44556764891297 
Median radius at 2 solar masses 13.79773989129582 
Lower and upper bound for quartile radius at 2 solar masses- 3.2560524210952018, 1.4724380608383658 
Median max mass 2.0398771477880606 
Median radius at median max mass 13.858821837453076 
Lower and upper bound for quartile radius at median max mass- 3.1380450659318164, 1.4629695185527591 
Median sound speed at 2x nuclear saturation density

{'pressure_2sat': array(3.16574253e+34),
 'p_lower_2sat': 2.65400195286234e+34,
 'p_upper_2sat': 1.547892560504343e+35,
 'pressure_6sat': array(4.83666258e+35),
 'p_lower_6sat': 4.649542032617266e+35,
 'p_upper_6sat': 1.9807267806850263e+36,
 'radius_1.4': array(12.94570439),
 'radius_lower_1.4': 3.94525781247831,
 'radius_upper_1.4': 1.44556764891297,
 'radius_2': array(13.79773989),
 'radius_lower_2': 3.2560524210952018,
 'radius_upper_2': 1.4724380608383658,
 'median_max_mass': 2.0398771477880606,
 'radius_max': array(13.85882184),
 'radius_lower_max': 3.1380450659318164,
 'radius_upper_max': 1.4629695185527591,
 'cs_2sat': array(0.14537894),
 'cs_lower_2sat': 0.1431299131454666,
 'cs_upper_2sat': 0.7814083051822388,
 'cs_6sat': array(0.42492387),
 'cs_lower_6sat': 0.41991829230465993,
 'cs_upper_6sat': 0.5663360319973965,
 'lambda': array(667.82494792),
 'lambda_lower': 621.5706902823191,
 'lambda_upper': 715.5602486998844}

In [31]:
diagnose_eos_set(cl_005)

For the set <HDF5 file "g.7_a1._cl.05.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 2.962203169902931e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.123281791755349e+34, 4.4639262611863205e+34 
Median pressure at 6x nuclear saturation density- 4.042897291326444e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 3.628369045028417e+35, 1.2187201632486042e+36 
Median radius at 1.4 solar masses 12.48256263188913 
Lower and upper bound for quartile radius at 1.4 solar masses- 2.910997412370996, 1.2553818250760127 
Median radius at 2 solar masses 12.905536009971296 
Lower and upper bound for quartile radius at 2 solar masses- 2.042810328183137, 1.3360153332817841 
Median max mass 1.7473784669810155 
Median radius at median max mass 12.697111616887558 
Lower and upper bound for quartile radius at median max mass- 2.5649791860219384, 1.342068283354811 
Median sound speed at 2x nuclear saturation dens

{'pressure_2sat': array(2.96220317e+34),
 'p_lower_2sat': 2.123281791755349e+34,
 'p_upper_2sat': 4.4639262611863205e+34,
 'pressure_6sat': array(4.04289729e+35),
 'p_lower_6sat': 3.628369045028417e+35,
 'p_upper_6sat': 1.2187201632486042e+36,
 'radius_1.4': array(12.48256263),
 'radius_lower_1.4': 2.910997412370996,
 'radius_upper_1.4': 1.2553818250760127,
 'radius_2': array(12.90553601),
 'radius_lower_2': 2.042810328183137,
 'radius_upper_2': 1.3360153332817841,
 'median_max_mass': 1.7473784669810155,
 'radius_max': array(12.69711162),
 'radius_lower_max': 2.5649791860219384,
 'radius_upper_max': 1.342068283354811,
 'cs_2sat': array(0.14906423),
 'cs_lower_2sat': 0.12215058160975426,
 'cs_upper_2sat': 0.3258317421563933,
 'cs_6sat': array(0.43563499),
 'cs_lower_6sat': 0.40623387324582516,
 'cs_upper_6sat': 0.5130262841752995,
 'lambda': array(478.48943364),
 'lambda_lower': 416.1604517860868,
 'lambda_upper': 477.09891919570765}

In [31]:
diagnose_eos_set(cl_01)

KeyError: 'Unable to synchronously open object (invalid identifier type to function)'

In [35]:
diagnose_eos_set(cl_sr01)

For the set <HDF5 file "g.7_a1._clsr.1.h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.055361095781963e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.0558047986553454e+34, 2.0536449459071502e+34 
Median pressure at 6x nuclear saturation density- 4.1822343691593854e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 2.943809437791394e+35, 3.4950129936748266e+35 
Median radius at 1.4 solar masses 12.097024530057837 
Lower and upper bound for quartile radius at 1.4 solar masses- 2.334794923120098, 1.2832364594665968 
Median radius at 2 solar masses 12.353707353944447 
Lower and upper bound for quartile radius at 2 solar masses- 1.1576100468468944, 1.0647221286831439 
Median max mass 1.7350508436455252 
Median radius at median max mass 12.124788485673806 
Lower and upper bound for quartile radius at median max mass- 1.7208517281867888, 1.289234019659192 
Median sound speed at 2x nuclear saturation

{'pressure_2sat': array(3.0553611e+34),
 'p_lower_2sat': 2.0558047986553454e+34,
 'p_upper_2sat': 2.0536449459071502e+34,
 'pressure_6sat': array(4.18223437e+35),
 'p_lower_6sat': 2.943809437791394e+35,
 'p_upper_6sat': 3.4950129936748266e+35,
 'radius_1.4': array(12.09702453),
 'radius_lower_1.4': 2.334794923120098,
 'radius_upper_1.4': 1.2832364594665968,
 'radius_2': array(12.35370735),
 'radius_lower_2': 1.1576100468468944,
 'radius_upper_2': 1.0647221286831439,
 'median_max_mass': 1.7350508436455252,
 'radius_max': array(12.12478849),
 'radius_lower_max': 1.7208517281867888,
 'radius_upper_max': 1.289234019659192,
 'cs_2sat': array(0.15681233),
 'cs_lower_2sat': 0.10755499037425595,
 'cs_upper_2sat': 0.10698087267815803,
 'cs_6sat': array(0.44495401),
 'cs_lower_6sat': 0.2767819448540313,
 'cs_upper_6sat': 0.21994594292240965,
 'lambda': array(361.98073361),
 'lambda_lower': 290.45380509925604,
 'lambda_upper': 403.90218644580557}

In [36]:
diagnose_eos_set(cl_1)

For the set <HDF5 file "g.7_a1._cl1..h5" (mode r)>: 
Median pressure at 2x nuclear saturation density- 3.072621670026849e+34 
Lower and upper bound for quartile pressure at 2x nuclear saturation density- 2.0667590696946762e+34, 1.9747012655130286e+34 
Median pressure at 6x nuclear saturation density- 4.249823565418301e+35 
Lower and upper bound for quartile pressure at 6x nuclear saturation density- 2.9386022306248525e+35, 2.7720850943454996e+35 
Median radius at 1.4 solar masses 12.072270973612836 
Lower and upper bound for quartile radius at 1.4 solar masses- 2.340378218362842, 1.2926377796906454 
Median radius at 2 solar masses 12.31058814321658 
Lower and upper bound for quartile radius at 2 solar masses- 0.9968953262233224, 1.0208766461125158 
Median max mass 1.7414295318901505 
Median radius at median max mass 12.114147406314885 
Lower and upper bound for quartile radius at median max mass- 1.6201353093030448, 1.2489625638191093 
Median sound speed at 2x nuclear saturation d

{'pressure_2sat': array(3.07262167e+34),
 'p_lower_2sat': 2.0667590696946762e+34,
 'p_upper_2sat': 1.9747012655130286e+34,
 'pressure_6sat': array(4.24982357e+35),
 'p_lower_6sat': 2.9386022306248525e+35,
 'p_upper_6sat': 2.7720850943454996e+35,
 'radius_1.4': array(12.07227097),
 'radius_lower_1.4': 2.340378218362842,
 'radius_upper_1.4': 1.2926377796906454,
 'radius_2': array(12.31058814),
 'radius_lower_2': 0.9968953262233224,
 'radius_upper_2': 1.0208766461125158,
 'median_max_mass': 1.7414295318901505,
 'radius_max': array(12.11414741),
 'radius_lower_max': 1.6201353093030448,
 'radius_upper_max': 1.2489625638191093,
 'cs_2sat': array(0.15788162),
 'cs_lower_2sat': 0.10829995353388175,
 'cs_upper_2sat': 0.09584065655042354,
 'cs_6sat': array(0.45369382),
 'cs_lower_6sat': 0.2692484912953056,
 'cs_upper_6sat': 0.15658238044644074,
 'lambda': array(355.36427708),
 'lambda_lower': 286.3398067678615,
 'lambda_upper': 397.4633628827974}

In [69]:

lam_sigmas_g7 = get_lambda_quantiles(eos_to_be_used, macro_g7, plot_masses, [50,95], cutoff = 30., verbose =  False)


In [70]:
int_func_l = interpolate.interp1d(plot_masses, lam_sigmas_g7[:,0])
median_lam = int_func_l(1.5)

In [71]:
median_lam


array(474.31859803)

In [86]:
mean_max_mass = get_max_mass_quantiles(eos_to_be_used, macro_g7, [50], cutoff = 30.)

In [89]:
mean_max_mass[0]


2.0398771477880606